## Preanalysis of the traffic data 

In [1]:
# import needed packages
import requests
import json
import pprint
import pandas as pd
from sqlalchemy import create_engine, text, inspect
from datetime import date

### Connection details for db and api 

In [2]:
ENDPOINT = 'traffic.cqznezykh98v.us-east-1.rds.amazonaws.com'
DB_NAME = 'traffic'
USERNAME = 'admin_user'
PASSWORD = 'KAjUGqLSwNhTrf6m58Ap'

In [3]:
# DB table name
table_name = 'miv_hourly'

# connection string for sql alchemy engine
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DB_NAME}'

# Package list of the Swiss open data portal
base_url = 'https://opendata.swiss/api/3/action/package_show?id='
# packages = 'https://opendata.swiss/api/3/action/package_list'
package = 'daten-der-verkehrszahlung-stundenwerte-seit-2012'

### request the url to request the csv files for the data of each year from the Open Data Swiss API

In [6]:
# Make the HTTP request
package_information = requests.get(base_url + package)

# Use the json module to load CKAN's response into a dictionary
package_dict = json.loads(package_information.content)

# Check the contents of the response.
assert package_dict['success'] is True  # again make sure if response is OK
package_dict = package_dict['result']  # we only need the 'result' part from the dictionary
# pprint.pprint(package_dict)  # pretty print the package information to screen

# Get the relevant metadata for the data from the dictionary
df_sources = pd.DataFrame({'year': [], 'filename': [], 'package_id': [], 'url': [], 'uri': [], 'format': []})

for ele in package_dict['resources']:
    to_append = [ele['url'][-8:-4], ele['url'][-25:-4], ele['package_id'], ele['url'], ele['uri'], ele['format']]
    df_sources.loc[len(df_sources)] = to_append

df_sources = df_sources[df_sources['format'] == "CSV"]  # only keep csv files in the dataframe
df_sources['year'] = df_sources['year'].astype(int)  # convert year to int
df_sources

,year,filename,package_id,url,uri,format
0,2014,hlung_miv_OD2031_2014,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
1,2018,hlung_miv_OD2031_2018,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
2,2020,hlung_miv_OD2031_2020,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
3,2012,hlung_miv_OD2031_2012,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
4,2021,hlung_miv_OD2031_2021,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
5,2016,hlung_miv_OD2031_2016,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
6,2015,hlung_miv_OD2031_2015,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
7,2022,hlung_miv_OD2031_2022,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
8,2017,hlung_miv_OD2031_2017,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV
9,2013,hlung_miv_OD2031_2013,f80fd59a-e52a-4caa-8b39-287662c0c517,https://data.stadt-zuerich.ch/dataset/sid_dav_...,https://ckan-ogdzh.clients.liip.ch/dataset/621...,CSV


### load data for all years in dataframe

In [9]:
df = pd.concat(map(lambda file: pd.read_csv(file), 
                   list(df_sources["url"])), ignore_index=True)
df

c:\users\larsn\appdata\local\programs\python\python39\lib\site-packages\pandas\core\reshape\concat.py:348: DtypeWarning: Columns (5,14) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


### determine the needed measuring locations

In [34]:
df.loc[df['Achse'] == "Schimmelstrasse"]["ZSID"].unique()

array(['Z068'], dtype=object)

In [35]:
df.loc[df['Achse'] == "Stampfenbachstrasse"]["ZSID"].unique()

array(['Z058'], dtype=object)

In [36]:
df.loc[df['Achse'] == "Rosengartenstrasse"]["ZSID"].unique()

array(['Z026', 'Z038'], dtype=object)

Above are the different measuring location ID's for the needed locations listed. The Rosengartenstrasse location has two different measuring stations. For our project the station with the id Z038 is used. 

### Delete the not needed measuring locations

In [38]:
df = df.loc[df['ZSID'].isin(["Z068", "Z058", "Z038"])]

### overview over columns and dtypes

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541572 entries, 613200 to 15129805
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   MSID                541572 non-null  object 
 1   MSName              541572 non-null  object 
 2   ZSID                541572 non-null  object 
 3   ZSName              541572 non-null  object 
 4   Achse               541572 non-null  object 
 5   HNr                 541572 non-null  object 
 6   Hoehe               541572 non-null  object 
 7   EKoord              541572 non-null  float64
 8   NKoord              541572 non-null  float64
 9   Richtung            541572 non-null  object 
 10  Knummer             541572 non-null  int64  
 11  Kname               541572 non-null  object 
 12  AnzDetektoren       541572 non-null  int64  
 13  D1ID                541572 non-null  int64  
 14  D2ID                541572 non-null  object 
 15  D3ID                541572 

In [59]:
df[df['AnzFahrzeuge'].isnull()]

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,...,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus
613200,Z038M001,Unbekannt,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,Unbekannt,2682070.59,1249909.12,Bucheggplatz,...,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,Unbekannt,Unbekannt,2014-01-01T00:00:00,2021-02-03,NaN,Fehlend
613201,Z038M001,Unbekannt,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,Unbekannt,2682070.59,1249909.12,Bucheggplatz,...,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,Unbekannt,Unbekannt,2014-01-01T01:00:00,2021-02-03,NaN,Fehlend
613202,Z038M001,Unbekannt,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,Unbekannt,2682070.59,1249909.12,Bucheggplatz,...,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,Unbekannt,Unbekannt,2014-01-01T02:00:00,2021-02-03,NaN,Fehlend
613203,Z038M001,Unbekannt,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,Unbekannt,2682070.59,1249909.12,Bucheggplatz,...,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,Unbekannt,Unbekannt,2014-01-01T03:00:00,2021-02-03,NaN,Fehlend
613204,Z038M001,Unbekannt,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,Unbekannt,2682070.59,1249909.12,Bucheggplatz,...,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,Unbekannt,Unbekannt,2014-01-01T04:00:00,2021-02-03,NaN,Fehlend
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15125624,Z068M002,Unbekannt,Z068,Schimmelstrasse (Werdstrasse),Schimmelstrasse,10,Unbekannt,2682054.00,1247168.00,Bahnhof Wiedikon,...,Schimmel-/Werdstr (Schulweg),1,12,Unbekannt,Unbekannt,Unbekannt,2019-07-10T19:00:00,2021-02-03,NaN,Fehlend
15125625,Z068M002,Unbekannt,Z068,Schimmelstrasse (Werdstrasse),Schimmelstrasse,10,Unbekannt,2682054.00,1247168.00,Bahnhof Wiedikon,...,Schimmel-/Werdstr (Schulweg),1,12,Unbekannt,Unbekannt,Unbekannt,2019-07-10T20:00:00,2021-02-03,NaN,Fehlend
15125626,Z068M002,Unbekannt,Z068,Schimmelstrasse (Werdstrasse),Schimmelstrasse,10,Unbekannt,2682054.00,1247168.00,Bahnhof Wiedikon,...,Schimmel-/Werdstr (Schulweg),1,12,Unbekannt,Unbekannt,Unbekannt,2019-07-10T21:00:00,2021-02-03,NaN,Fehlend
15125627,Z068M002,Unbekannt,Z068,Schimmelstrasse (Werdstrasse),Schimmelstrasse,10,Unbekannt,2682054.00,1247168.00,Bahnhof Wiedikon,...,Schimmel-/Werdstr (Schulweg),1,12,Unbekannt,Unbekannt,Unbekannt,2019-07-10T22:00:00,2021-02-03,NaN,Fehlend


Only the AnzFahrzeuge column contains null values. The measuring for these timepoints is missing, which is indicated by the AnzFahrzeugeStatus column.

### determine not needed or informationless columns by analysing the unique values of a column

In [40]:
df.nunique()

MSID                      6
MSName                    1
ZSID                      3
ZSName                    3
Achse                     3
HNr                       5
Hoehe                     1
EKoord                    6
NKoord                    6
Richtung                  6
Knummer                   3
Kname                     3
AnzDetektoren             2
D1ID                      5
D2ID                      6
D3ID                      1
D4ID                      1
MessungDatZeit        90253
LieferDat                 4
AnzFahrzeuge           2324
AnzFahrzeugeStatus        3
dtype: int64

In [42]:
df.HNr.unique()

array(['23', 56, 10, '56', '10'], dtype=object)

HNr has technically only three unique values, but because of different datatypes 5 unique values are listed. This is not an issue and will be handled in the etl process.

In [43]:
df.Hoehe.unique()

array(['Unbekannt'], dtype=object)

In [44]:
df.MSName.unique()

array(['Unbekannt'], dtype=object)

In [50]:
df.D3ID.unique()

array(['Unbekannt'], dtype=object)

In [51]:
df.D4ID.unique()

array(['Unbekannt'], dtype=object)

all of the above columns have only one unique value unknown. Therefore these columns offer no valuable information for our project and can be excluded in the etl process.

In [52]:
df.LieferDat.unique()

array(['2021-02-03', '2021-02-04', '2021-12-30', '2022-04-21'],
      dtype=object)

the column LieferDat is not relevant for our project and can be excluded in the etl process.